In [0]:
import requests
from io import BytesIO
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator




In [0]:
#link para download
url = "https://query.data.world/s/ts3253q4zig6243pl6llh4wh6da2i5?dws=00000"

#fazendo a requisição para download
response = requests.get(url)
response.raise_for_status()

#caminho local onde será salvo
dbfs_path = "/dbfs/religions.csv"


#criando arquivo com o retorno da requisição
with open(dbfs_path, 'wb') as f:
    f.write(response.content)

In [0]:
%sh
wget https://api.worldbank.org/v2/en/indicator/SP.DYN.TFRT.IN?downloadformat=csv
unzip 'SP.DYN.TFRT.IN?downloadformat=csv'



--2023-09-25 21:46:56--  https://api.worldbank.org/v2/en/indicator/SP.DYN.TFRT.IN?downloadformat=csv
Resolving api.worldbank.org (api.worldbank.org)... 52.188.136.184
Connecting to api.worldbank.org (api.worldbank.org)|52.188.136.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69770 (68K) [application/zip]
Saving to: ‘SP.DYN.TFRT.IN?downloadformat=csv’

     0K .......... .......... .......... .......... .......... 73%  634K 0s
    50K .......... ........                                   100% 82.4M=0.08s

2023-09-25 21:46:56 (861 KB/s) - ‘SP.DYN.TFRT.IN?downloadformat=csv’ saved [69770/69770]

Archive:  SP.DYN.TFRT.IN?downloadformat=csv
  inflating: Metadata_Indicator_API_SP.DYN.TFRT.IN_DS2_en_csv_v2_5871756.csv  
  inflating: API_SP.DYN.TFRT.IN_DS2_en_csv_v2_5871756.csv  
  inflating: Metadata_Country_API_SP.DYN.TFRT.IN_DS2_en_csv_v2_5871756.csv  


In [0]:
%sh
mv API_SP.DYN* /dbfs/birth_rate.csv

In [0]:
%sh
tail -n +5 /dbfs/birth_rate.csv > arquivo_temp.csv && mv arquivo_temp.csv /dbfs/birth_rate.csv

In [0]:
dbutils.fs.cp("file:/dbfs/religions.csv", "dbfs:/FileStore/religions.csv")
dbutils.fs.cp("file:/dbfs/birth_rate.csv", "dbfs:/FileStore/birth_rate.csv")



Out[110]: True

In [0]:
religions = spark.read.option("header","true").csv("dbfs:/FileStore/religions.csv")

In [0]:
birth_rate = spark.read.option("header","true").csv("dbfs:/FileStore/birth_rate.csv")

In [0]:
taxa_62_anos = (religions
                                     .join(birth_rate.alias("b"),
                                           religions['Country or Area'] == birth_rate['Country Name'])
                                     .withColumn("maior_taxa_por_pais_range_62_anos",
                                    ((col("1961") + col("1962") + col("1963") + col("1964") + col("1965") + col("1966") + col("1967")
                                    + col("1968") + col("1969") + col("1970") + col("1971") + col("1972") + col("1973") + col("1974")
                                    + col("1975") + col("1976") + col("1977") + col("1978") + col("1979") + col("1980") + col("1981")
                                    + col("1982") + col("1983") + col("1984") + col("1985") + col("1986") + col("1987") + col("1988")
                                    + col("1989") + col("1990") + col("1991") + col("1992") + col("1993") + col("1994") + col("1995")
                                    + col("1996") + col("1997") + col("1998") + col("1999") + col("2000") + col("2001") + col("2002")
                                    + col("2003") + col("2004") + col("2005") + col("2006") + col("2007") + col("2008") + col("2009")
                                    + col("2010") + col("2011") + col("2012") + col("2013") + col("2014") + col("2015") + col("2016")
                                    + col("2017") + col("2018") + col("2019") + col("2020") + col("2021"))/62))
                                     .select("Country or Area","maior_taxa_por_pais_range_62_anos"))

grouped_data = (taxa_62_anos.groupBy("Country or Area")
                .agg(avg("maior_taxa_por_pais_range_62_anos").alias("maior_taxa_por_pais_range_62_anos"))
                .sort(col("maior_taxa_por_pais_range_62_anos").desc())
                .select("Country or Area", "maior_taxa_por_pais_range_62_anos"))

grouped_data.display()

Country or Area,maior_taxa_por_pais_range_62_anos
Niger,7.449145161290324
Burundi,6.711516129032252
Uganda,6.531612903225806
Rwanda,6.4640161290322595
Burkina Faso,6.359532258064517
Ethiopia,6.324145161290322
Malawi,6.263016129032268
Zambia,6.212612903225814
Kenya,5.960177419354836
Mozambique,5.920370967741951


In [0]:
maior_taxa = (religions
                                     .join(birth_rate.alias("b"),
                                           religions['Country or Area'] == birth_rate['Country Name'])
                                     .withColumn("maior_taxa",
                                    ((col("1961") + col("1962") + col("1963") + col("1964") + col("1965"))/5))
                                     .select("Country or Area","maior_taxa"))


grouped_data_grouped = (maior_taxa.groupBy("Country or Area")
                .agg(avg("maior_taxa").alias("maior_taxa"))
                .sort(col("maior_taxa").desc())
                .select("Country or Area", "maior_taxa"))

grouped_data_grouped.display()

Country or Area,maior_taxa
Marshall Islands,8.306799999999997
Rwanda,8.197000000000001
Kenya,7.8666
Samoa,7.652199999999995
Niger,7.489600000000001
Mongolia,7.312199999999995
Zambia,7.2394
Malawi,7.098799999999992
Nicaragua,7.097600000000006
Bahrain,7.0688


In [0]:
indice_religioes = (religions
                                     .join(birth_rate,
                                           religions['Country or Area'] == birth_rate['Country Name'])
                                     .withColumn("indice_religiao",
                                    ((col("1961") + col("1962") + col("1963") + col("1964") + col("1965")
                                     +col("1966") + col("1967") + col("1968") + col("1969") + col("1970")
                                     +col("1971") + col("1972") + col("1973") + col("1974") + col("1975"))/15))
                                     .select("Religion","indice_religiao"))

indice_religioes_grouped = (indice_religioes.groupBy("Religion")
                .agg(avg("indice_religiao").alias("indice_religiao"))
                .sort(col("indice_religiao").desc())
                .select("Religion", "indice_religiao"))

indice_religioes_grouped.display()

Religion,indice_religiao
Worship Centre,7.4110000000000005
Peace Chapel,7.4110000000000005
Voice of Christ,7.4110000000000005
Samoa Evangelism,7.4110000000000005
Community Church,7.4110000000000005
CCCJS,7.4110000000000005
Aoga Tusi Paia,7.4110000000000005
Shaman,7.379666666666667
Traditional,6.644759999999995
Badimo,6.627866666666668


In [0]:
indice_religioes_5anos = (religions
                                     .join(birth_rate,
                                           religions['Country or Area'] == birth_rate['Country Name'])
                                     .withColumn("indice_religiao",
                                    ((col("1961") + col("1962") + col("1963") + col("1964") + col("1965"))/5))
                                     .select("Religion","indice_religiao"))

indice_religioes_5anos_grouped = (indice_religioes_5anos.groupBy("Religion")
                .agg(avg("indice_religiao").alias("indice_religiao"))
                .sort(col("indice_religiao").desc())
                .select("Religion", "indice_religiao"))

indice_religioes_5anos_grouped.display()

Religion,indice_religiao
Worship Centre,7.652199999999998
Peace Chapel,7.652199999999998
Voice of Christ,7.652199999999998
Samoa Evangelism,7.652199999999998
Community Church,7.652199999999998
CCCJS,7.652199999999998
Aoga Tusi Paia,7.652199999999998
Shaman,7.3122
Iglesia Evangelista Methodista en Las Filipinas,6.9403999999999995
Philippine Benevolent Missionaries Association,6.9403999999999995


In [0]:
menor_taxa_62_anos = (religions
                                     .join(birth_rate.alias("b"),
                                           religions['Country or Area'] == birth_rate['Country Name'])
                                     .withColumn("menor_taxa_por_pais_range_62_anos",
                                    ((col("1961") + col("1962") + col("1963") + col("1964") + col("1965") + col("1966") + col("1967")
                                    + col("1968") + col("1969") + col("1970") + col("1971") + col("1972") + col("1973") + col("1974")
                                    + col("1975") + col("1976") + col("1977") + col("1978") + col("1979") + col("1980") + col("1981")
                                    + col("1982") + col("1983") + col("1984") + col("1985") + col("1986") + col("1987") + col("1988")
                                    + col("1989") + col("1990") + col("1991") + col("1992") + col("1993") + col("1994") + col("1995")
                                    + col("1996") + col("1997") + col("1998") + col("1999") + col("2000") + col("2001") + col("2002")
                                    + col("2003") + col("2004") + col("2005") + col("2006") + col("2007") + col("2008") + col("2009")
                                    + col("2010") + col("2011") + col("2012") + col("2013") + col("2014") + col("2015") + col("2016")
                                    + col("2017") + col("2018") + col("2019") + col("2020") + col("2021"))/62))
                                     .select("Country or Area","menor_taxa_por_pais_range_62_anos"))

menor_taxa_62_anos_grouped_data = (menor_taxa_62_anos.groupBy("Country or Area")
                .agg(avg("menor_taxa_por_pais_range_62_anos").alias("menor_taxa_por_pais_range_62_anos"))
                .sort(col("menor_taxa_por_pais_range_62_anos").asc())
                .select("Country or Area", "menor_taxa_por_pais_range_62_anos"))

menor_taxa_62_anos_grouped_data.display()

Country or Area,menor_taxa_por_pais_range_62_anos
Palau,null
Cayman Islands,null
Liechtenstein,null
Seychelles,null
Germany,1.5572580645161282
Hungary,1.6551612903225694
Switzerland,1.658387096774197
Serbia,1.6602419354838773
Austria,1.6811290322580694
Croatia,1.6851612903225826


In [0]:
menor_taxa_20_anos = (religions
                                     .join(birth_rate.alias("b"),
                                           religions['Country or Area'] == birth_rate['Country Name'])
                                     .withColumn("menor_taxa_por_pais_range_20_anos",
                                    ((col("1961") + col("1962") + col("1963") + col("1964") + col("1965") + col("1966") + col("1967")
                                    + col("1968") + col("1969") + col("1970") + col("1971") + col("1972") + col("1973") + col("1974")
                                    + col("1975"))/15))
                                     .select("Country or Area","menor_taxa_por_pais_range_20_anos"))

menor_taxa_20_anos_grouped_data = (menor_taxa_20_anos.groupBy("Country or Area")
                .agg(avg("menor_taxa_por_pais_range_20_anos").alias("menor_taxa_por_pais_range_20_anos"))
                .sort(col("menor_taxa_por_pais_range_20_anos").asc())
                .select("Country or Area", "menor_taxa_por_pais_range_20_anos"))

menor_taxa_20_anos_grouped_data.display()

Country or Area,menor_taxa_por_pais_range_20_anos
Palau,null
Cayman Islands,null
Liechtenstein,null
Seychelles,null
Serbia,1.9299333333333368
Hungary,1.9719999999999909
Estonia,2.0153333333333525
Croatia,2.069933333333327
Finland,2.1213333333333284
Germany,2.1466666666666665


In [0]:
(religions.groupBy("Country or Area")
          .agg(count("Religion").alias("Total_de_religioes_distintas"))
          .sort(col("Total_de_religioes_distintas").desc())
          .select("Country or Area","Total_de_religioes_distintas").limit(1).display())

Country or Area,Total_de_religioes_distintas
Canada,1535


In [0]:
(religions.groupBy("Country or Area")
          .agg(count("Religion").alias("Total_de_religioes_distintas"))
          .sort(col("Total_de_religioes_distintas").asc())
          .select("Country or Area","Total_de_religioes_distintas").limit(1).display())

Country or Area,Total_de_religioes_distintas
Niger,7


In [0]:
# Criando um dataframe de exemplo
data = [(50, 200000), (60, 220000), (80, 270000), (100, 320000), (120, 380000)]
df = spark.createDataFrame(data, ["area", "price"])

# Exibindo os dados
df.show()


+----+------+
|area| price|
+----+------+
|  50|200000|
|  60|220000|
|  80|270000|
| 100|320000|
| 120|380000|
+----+------+



In [0]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=1234)


In [0]:
# Transformando os dados para o formato correto
vectorAssembler = VectorAssembler(inputCols=["area"], outputCol="features")
train_data = vectorAssembler.transform(train_data)
test_data = vectorAssembler.transform(test_data)

# Definindo e treinando o modelo de regressão linear
lr = LinearRegression(featuresCol='features', labelCol='price')
lr_model = lr.fit(train_data)


In [0]:
training_summary = lr_model.summary
print(f"R2: {training_summary.r2}")
print(f"RMSE: {training_summary.rootMeanSquaredError}")


R2: 0.9969905949059491
RMSE: 3612.309578068418


In [0]:
predictions = lr_model.transform(test_data)
predictions.select("prediction", "price", "features").show()


+----------+-----+--------+
|prediction|price|features|
+----------+-----+--------+
+----------+-----+--------+



**Sugestões para Melhoria**

***Feature Engineering***: Poderíamos adicionar mais recursos, como o número de quartos, a localização, entre outros.

***Regularização***: Utilizar técnicas de regularização como Lasso ou Ridge para evitar overfitting, especialmente útil se tivermos muitos recursos.

***Validação Cruzada***: Usar validação cruzada para otimizar os hiperparâmetros do modelo.

In [0]:
# Definindo a grid de parâmetros
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Definindo a validação cruzada
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="price"),
                          numFolds=3)

# Treinando o modelo com validação cruzada
cv_model = crossval.fit(train_data)

# Fazendo previsões com o melhor modelo
cv_predictions = cv_model.transform(test_data)
cv_predictions.select("prediction", "price", "features").show()


+----------+-----+--------+
|prediction|price|features|
+----------+-----+--------+
+----------+-----+--------+

